In [ ]:
!pip install rouge tf-estimator-nightly==2.8.0.dev2021122109 simplet5 folium==0.2.1 -q

In [ ]:
# Imports

import numpy as np 
import pandas as pd 
import nltk
from random import sample
import torch
torch.cuda.empty_cache()
import os
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from random import sample
from rouge import Rouge

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATA_PATH = '/content/drive/MyDrive/cnn_dailymail'
path = os.path.join(DATA_PATH, 'train.csv')
training_df = pd.read_csv(path, engine='python', error_bad_lines=False)

MAX_LEN = 512
SUMMARY_LEN = 150
TRAINING_SIZE = 5000

training_df = training_df.iloc[0:TRAINING_SIZE,:].copy()
training_article_ls = list(training_df['article'])
training_highlight_ls = list(training_df['highlights'])

df = pd.DataFrame(columns=['target_text','source_text'])
df['target_text'] = training_highlight_ls
df['source_text'] = ['summarize: '+item for item in training_article_ls]

In [ ]:
model = simplet5.SimpleT5()

model.from_pretrained(model_type="t5", model_name="t5-base")

MAX_EPOCHS = 10

torch.cuda.memory_summary(device=None, abbreviated=False)

model.train(
    train_df = df[0:(int)(0.7*TRAINING_SIZE)],
    eval_df = df[(int)(0.7*TRAINING_SIZE):TRAINING_SIZE], 
    source_max_token_len = MAX_LEN, 
    target_max_token_len = SUMMARY_LEN, 
    batch_size = 2, 
    max_epochs = MAX_EPOCHS, 
    use_gpu = True
    )
            
model_path = ''
rootdir = 'outputs/'
for it in os.scandir(rootdir):
    if it.is_dir():
        if 't5-epoch-' + (str)(MAX_EPOCHS-1) in it.path:
            model_path = it.path
            print(model_path)

In [ ]:
# Evaluation

# UNCOMMENT IF RUNNING ONLY EVALUATION ON OUTPUT SAMPLES
# model.load_model("t5","./" + model_path , use_gpu=True)

tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Sampling on 10 articles
for index in sample(list(np.arange(len(training_articles))), 10):
    rouge = Rouge()
    prediction = model.predict(training_article_ls[index])

    r_score = rouge.get_scores(prediction, training_highlight_ls[index])